In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.4/455.4 MB 11.3 MB/s  0:00:400m eta 0:00:010:00:02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-4.1.1-py2.py3-none-any.whl size=456008705 sha256=40018d987fb26bb18ee8cecfe22944d969e5b7412f07b6bbfe56e72b4fdf6e10
  Stored in directory: /home/pc/.cache/pip/wheels/f4/ca/ea/203f40b3e935bbf99bee851c2f4a87d22996ab8212d367ce58
Successfully built pyspark
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pyspark]━━━ 1/2 [pyspark]


# Question 1

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()


print(f"The spark version: {spark.version}")

26/01/22 08:58:54 WARN Utils: Your hostname, DESKTOP-G33DRVE resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/22 08:58:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/22 08:58:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


The spark version: 3.5.0


# Question 2

In [2]:

# Read the dataset
file_path = "yellow_tripdata_2024-10.parquet"
df = spark.read.parquet(file_path)

# repartition and save as Parquet
output_path = "output/yellow_tripdata_partitioned"
df.repartition(4).write.mode("overwrite").parquet(output_path)


# Calculate the average size of Parquet files
import os

file_sizes = [os.path.getsize(os.path.join(output_path, f)) for f in os.listdir(output_path) if f.endswith(".parquet")]
avg_size_mb = sum(file_sizes) / len(file_sizes) / (1024 * 1024)
print(f"Average Parquet File Size: {avg_size_mb:.2f} MB")


[Stage 3:>                                                          (0 + 4) / 4]

Average Parquet File Size: 22.39 MB


# Question 3

In [3]:

from pyspark.sql.functions import col, to_date

# Filter trips that started on October 15th
df_filtered = df.filter(col("tpep_pickup_datetime").substr(1, 10) == "2024-10-15")

print("Trips on October 15:", df_filtered.count())

Trips on October 15: 128893


# Question 4

In [4]:
# Longest trip

df.registerTempTable('trips_data')
spark.sql("""
select MAX(timestampdiff(HOUR, tpep_pickup_datetime, tpep_dropoff_datetime))
from trips_data
""").show()

/home/pc/spark/spark-3.5.0/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+---------------------------------------------------------------------+
|max(timestampdiff(HOUR, tpep_pickup_datetime, tpep_dropoff_datetime))|
+---------------------------------------------------------------------+
|                                                                  162|
+---------------------------------------------------------------------+



# Question 5

In [5]:

# Spark UI Port
print("Spark UI runs on port: 4040")

Spark UI runs on port: 4040


In [6]:

!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2026-01-22 08:59:22--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.84.167.134, 52.84.167.55, 52.84.167.2, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.84.167.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.001s  

2026-01-22 08:59:23 (11.7 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



# Question 6

In [7]:
# Least frequent pickup location zone
zone_lookup = spark.read.csv("taxi_zone_lookup.csv", header=True, inferSchema=True)
df.createOrReplaceTempView("trips")
zone_lookup.createOrReplaceTempView("zones")

least_frequent_zone = spark.sql("""
    SELECT zones.Zone, COUNT(*) as trip_count
    FROM trips
    JOIN zones ON trips.PULocationID = zones.LocationID
    GROUP BY zones.Zone
    ORDER BY trip_count ASC
    LIMIT 1
""").collect()[0][0]

print("Least Frequent Pickup Location Zone:", least_frequent_zone)

Least Frequent Pickup Location Zone: Governor's Island/Ellis Island/Liberty Island
